# LangChain 모듈 - Model I/O
Language Model 애플리케이션의 핵심 요소는 Model입니다.   
LangChain은 언어모델(Language Model)과 인터페이스할 수 있는 빌딩블록을 제공합니다.
    
- Code 출처 : https://python.langchain.com/docs/modules/model_io/
- 수정사항 : 설명과 프롬프트 내용을 한글로 변경 

In [1]:
%pip install -qU langchain langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [2]:
# import os
# os.environ["OPENAI_API_KEY"] = "<your OpenAI API key if not set as env var>"

### Models
- LangChain이 통합하는 모델 유형으로 LLM과 ChatModel이 있습니다.
- LLM 객체는 문자열을 입력으로 받고 문자열을 반환합니다.
- ChatModel 객체는 메시지 리스트를 입력으로 받고 메시지 리스트를 반환합니다.

In [3]:
from langchain_openai import OpenAI
from langchain_openai import ChatOpenAI

llm = OpenAI()
chat_model = ChatOpenAI()

In [4]:
from langchain.schema import HumanMessage

text = "컬러풀한 양말을 만드는 회사의 회사명으로 무엇이 좋을까?"
messages = [HumanMessage(content=text)]

In [5]:
llm.invoke(text)

'\n\n1. "Rainbow Socks Co."\n2. "Vibrant Hues Socks Inc."\n3. "Colorful Threads Co."\n4. "Spectrum Socks Ltd."\n5. "Rainbow Footwear Co."\n6. "Bright Steps Inc."\n7. "Chromatic Sox Co."\n8. "Hueful Socks Inc."\n9. "Vivid Sock Company"\n10. "Color Craze Socks Co."'

In [6]:
chat_model.invoke(messages)

AIMessage(content='1. 레인보우 삭스\n2. 브라이트 풋위어\n3. 컬러풀 토시즈\n4. 레인보우 스타일 삭스\n5. 하피 피트 삭스\n6. 브라이트 삭스 컴퍼니\n7. 레인보우 피트웨어\n8. 컬러풀 소크스\n9. 레인보우 스펙트럼 삭스\n10. 하피 피트 컬렉션', response_metadata={'token_usage': {'completion_tokens': 153, 'prompt_tokens': 41, 'total_tokens': 194}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_4f0b692a78', 'finish_reason': 'stop', 'logprobs': None})

### Prompt Templates
- 대부분의 LLM 애플리케이션은 사용자 입력을 LLM에 직접 전달하지 않습니다.
- Prompt Template을 사용하여 입력 텍스트와 특정 작업에 대한 추가 컨텍스트를 제공합니다.  
- 기본적으로 PromptTemplate은 템플릿 작성에 Python의 str.format 구문 {} 을 사용합니다.

In [7]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    "{content}에 대한 {adjective} 농담을 말해봐."
)

prompt_template.format( content="치킨", adjective="웃끼는",)


'치킨에 대한 웃끼는 농담을 말해봐.'

템플릿은 변수가 없는 경우를 포함하여 원하는 갯수의 변수를 지원합니다:

In [8]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template("농담을 해보세요")
prompt_template.format()

'농담을 해보세요'

- Prompt Template은 메시지 리스트를 만드는 데에도 사용할 수 있습니다.

### ChatPromptTemplate
- Chat Model 입력은 채팅 메시지 리스트입니다.
- 각 채팅 메시지는 콘텐츠 및 역할이라는 추가 매개변수와 연결됩니다.
- 예를 들어 OpenAI Chat Completions API에서 채팅 메시지는 AI assistant, human, system role에 연결될 수 있습니다.

In [9]:
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 유용한 AI 봇입니다. 당신의 이름은 {name} 입니다."),
        ("human", "안녕하세요, 어떻게 지내세요?"),
        ("ai", "잘 지내고 있어요, 고마워요!"),
        ("human", "{user_input}"),
    ]
)

messages = chat_template.format_messages(name="민수", user_input="너의 이름은?")
messages

[SystemMessage(content='당신은 유용한 AI 봇입니다. 당신의 이름은 민수 입니다.'),
 HumanMessage(content='안녕하세요, 어떻게 지내세요?'),
 AIMessage(content='잘 지내고 있어요, 고마워요!'),
 HumanMessage(content='너의 이름은?')]

In [10]:
# Get a chat completion from the formatted messages
model = ChatOpenAI()
result = model.invoke(messages)
print(result)

content='제 이름은 민수입니다. 궁금한 게 있으면 언제든지 물어보세요!' response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 84, 'total_tokens': 114}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_fa89f7a861', 'finish_reason': 'stop', 'logprobs': None}


- ChatPromptTemplate.from_messages는 다양한 메시지 표현을 허용합니다.
- 예를 들어 위에서 사용한 (type, content)의 2-tuple 표현을 사용하는 것 외에도   
   MessagePromptTemplate 또는 BaseMessage의 인스턴스를 전달할 수 있습니다.

In [11]:
from langchain.prompts import HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage
from langchain_openai import ChatOpenAI


chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=(
                "당신은 사용자의 입력 문장을 아주 멋있게 긴 장문으로 새로 작성하는 소설작가입니다."
            )
        ),
        HumanMessagePromptTemplate.from_template("{text}"),
    ]
)

messages = chat_template.format_messages(
    text="하늘은 파랗게 구름은 하얗게 실바람도 불어와 부풀은 내 마음"
)

print(messages)

[SystemMessage(content='당신은 사용자의 입력 문장을 아주 멋있게 긴 장문으로 새로 작성하는 소설작가입니다.'), HumanMessage(content='하늘은 파랗게 구름은 하얗게 실바람도 불어와 부풀은 내 마음')]


In [12]:
# Get a chat completion from the formatted messages
model = ChatOpenAI()
result = model.invoke(messages)
print(result)

content='그 날, 너와 함께였던 그 어느 날, 하늘은 맑고 파랗게 펼쳐져 있었다. 그 위를 지나가는 구름들은 하얗고 부드러웠다. 바람은 가볍게 스쳐가며 나의 마음을 부풀게 했다. 내 안에는 당신과 함께한 그 순간의 감정이 넘쳐흘렀다. 햇살은 부드럽게 내 얼굴을 쓸어주었고, 마음은 평화롭게 느껴졌다. 그 순간, 세상의 모든 것이 조용해지고, 오롯이 우리 둘만이 존재하는 것처럼 느껴졌다. 바람은 내 머릿결을 쓸어주며, 마음은 신비한 울림으로 가득 차 있었다. 이 순간을 영원히 간직하고 싶었다. 그 순간, 우리의 마음은 하늘과 구름처럼 맑고 푸르게 펼쳐져 있었고, 실바람처럼 가슴 속을 가볍게 타고 지나가는 것을 느꼈다. 함께 한 그 순간이 나에게는 소중한 보물이 되었다. 그리고 그 순간은 영원히 나의 마음속에 새겨져 남을 것이다.' response_metadata={'token_usage': {'completion_tokens': 432, 'prompt_tokens': 83, 'total_tokens': 515}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_fa89f7a861', 'finish_reason': 'stop', 'logprobs': None}


### Chat prompt 구성
- LangChain은 프롬프트의 여러 부분을 함께 구성할 수 있는 사용자 친화적인 인터페이스를 제공합니다.
- 문자열 프롬프트 또는 채팅 프롬프트를 사용하여 이 작업을 수행할 수 있습니다.
- 이러한 방식으로 프롬프트를 구성하면 구성 요소를 쉽게 재사용할 수 있습니다.  

#### String prompt 구성
- 문자열 프롬프트로 작업할 때 각 템플릿이 함께 결합됩니다.  
- 프롬프트를 직접 사용하거나 문자열을 사용하여 작업할 수 있습니다(리스트의 첫 번째 요소는 프롬프트여야 함).

In [13]:
from langchain.prompts import PromptTemplate

prompt = (
    PromptTemplate.from_template("{topic}에 대한 농담을 들려주세요.")
    + ", 재미있게 만드세요."
    + "\n\ {language} 언어로 해주세요."
)

In [14]:
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI

model = ChatOpenAI()
chain = LLMChain(llm=model, prompt=prompt)
chain.run(topic="sports", language="korean")

d:\강의자료\생성형AI로AI애플리케이션개발하기\py311\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'왜 골프 선수들은 항상 우산을 가져다 놓을까요? \n\n- 왜냐하면 그들이 항상 퍼팅을 연습하고 있기 때문에, 물이 그들을 따라다니기 때문이죠!'

#### Chat prompt 구성
- Chat prompt 메시지 리스트로 구성됩니다
- 개발자 경험을 위해 이러한 프롬프트를 생성하는 편리한 방법을 추가 되었습니다.
- 이 파이프라인에서 각각의 새 요소는 최종 프롬프트의 새 메시지입니다.

In [15]:
from langchain.schema import AIMessage, HumanMessage, SystemMessage

LangChain에는 역할에 따라 다른 Message 클래스가 있습니다.
- HumanMessage : 사용자가 보낸 메시지
- AIMessage : 모델의 메시지
- SystemMessage : 모델에 동작 방법을 알려줍니다. 
- FunctionMessage : 함수 호출의 결과를 나타냅니다
- ToolMessage : 도구 호출의 결과를 나타냅니다.  
 
먼저 SystemMessage로 기본 ChatPromptTemplate을 초기화해 보겠습니다.  
SystemMessage로 시작할 필요는 없지만 다음과 같이 시작하는 것이 좋습니다.

In [16]:
prompt = SystemMessage(content="너는 좋은 해적이야.")

그런 다음 다른 메시지 또는 메시지 템플릿과 결합하여 파이프라인을 쉽게 만들 수 있습니다.   
서식을 지정할 변수가 없는 경우에는 메시지를 사용하고, 서식을 지정할 변수가 있는 경우에는 메시지템플릿을 사용합니다.   
문자열만 사용할 수도 있습니다(참고: 이 경우 자동으로 휴먼메시지프롬프트템플릿으로 유추됩니다.).

In [17]:
new_prompt = (
    prompt + HumanMessage(content="안녕") + AIMessage(content="뭐라고?") + "{input}"
)

내부적으로는 ChatPromptTemplate 클래스의 인스턴스가 생성되므로 이전과 마찬가지로 사용할 수 있습니다.

In [18]:
new_prompt.format_messages(input="안녕이라고 말했어")

[SystemMessage(content='너는 좋은 해적이야.'),
 HumanMessage(content='안녕'),
 AIMessage(content='뭐라고?'),
 HumanMessage(content='안녕이라고 말했어')]

 결과
이전과 마찬가지로 LLMChain에서도 사용할 수 있습니다.

In [19]:
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI

model = ChatOpenAI()
chain = LLMChain(llm=model, prompt=new_prompt)
chain.run("안녕이라고 말했어")

'미안해, 내가 좀 혼란스러웠나봐. 안녕! 어떻게 도와줄까?'

### ChatModel
- ChatModel은 LangChain의 핵심 구성 요소입니다.  
- LangChain은 OpenAI, Cohere, Hugging Face 등에서 제공하는 많은 Model과 상호 작용할 수 있는 표준 인터페이스를 제공합니다.
- LangChain을 사용하면 동기, 비동기, 배치(batch) 및 스트리밍 모드에서 모델을 사용할 수 있으며 캐싱 둥 기타 기능을 제공합니다.

In [20]:
from langchain_openai import ChatOpenAI

chat = ChatOpenAI()

In [21]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="You're a helpful assistant"),
    HumanMessage(content="모델 정규화의 목적은 무엇인가요?"),
]

In [22]:
chat.invoke(messages)

AIMessage(content='모델 정규화의 주요 목적은 모델의 과적합을 방지하고 일반화 성능을 향상시키는 것입니다. 모델이 훈련 데이터에 지나치게 적합되어 새로운 데이터에 대해 부정확하게 예측하는 과적합을 방지하기 위해 모델의 복잡도를 제어하고 최적화하는 것이 중요합니다. 따라서 모델 정규화는 모델의 일반화 능력을 향상시키고 예측 성능을 향상시킬 수 있습니다.', response_metadata={'token_usage': {'completion_tokens': 176, 'prompt_tokens': 37, 'total_tokens': 213}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_4f0b692a78', 'finish_reason': 'stop', 'logprobs': None})

In [23]:
for chunk in chat.stream(messages):
    print(chunk.content, end="", flush=True)

모델 정규화의 주요 목적은 과적합(overfitting)을 방지하고 모델의 일반화 성능을 향상시키는 것입니다. 모델이 훈련 데이터에 너무 맞추어져서 새로운 데이터에 대한 예측 능력이 떨어지는 과적합 문제를 해결하기 위해 모델의 복잡성을 줄이고 일반화 성능을 향상시키는 것이 모델 정규화의 핵심 목적입니다.

In [24]:
chat.batch([messages])

[AIMessage(content='모델 정규화의 주요 목적은 모델의 일반화 성능을 향상시키는 것입니다. 모델 정규화는 과적합을 방지하고 모델의 안정성을 높이는데 도움이 됩니다. 이를 통해 모델이 새로운 데이터에 대해 더 일반화된 예측을 할 수 있게 됩니다. 종종 모델 정규화는 모델의 복잡성을 줄이고 학습 데이터에 대한 의존성을 줄여서 모델의 성능을 향상시키는 데 도움이 됩니다.', response_metadata={'token_usage': {'completion_tokens': 186, 'prompt_tokens': 37, 'total_tokens': 223}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_fa89f7a861', 'finish_reason': 'stop', 'logprobs': None})]

In [25]:
await chat.ainvoke(messages)

AIMessage(content='모델 정규화의 주요 목적은 과적합(overfitting)을 방지하고 모델의 일반화 성능을 향상시키는 것입니다. 이를 통해 모델이 학습 데이터에 너무 의존하여 새로운 데이터에 대한 예측 능력이 떨어지는 문제를 해결할 수 있습니다. 또한 모델 정규화는 변수 간의 관계를 조절하고 더 간결한 모델을 만들어줌으로써 모델의 해석력을 향상시킬 수도 있습니다. 종류로는 L1 정규화(라쏘), L2 정규화(릿지), ElasticNet 등이 있습니다.', response_metadata={'token_usage': {'completion_tokens': 210, 'prompt_tokens': 37, 'total_tokens': 247}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_4f0b692a78', 'finish_reason': 'stop', 'logprobs': None})

In [26]:
async for chunk in chat.astream(messages):
    print(chunk.content, end="", flush=True)

모델 정규화의 주요 목적은 과적합(overfitting)을 방지하고 모델의 일반화 성능을 향상시키는 것입니다. 모델이 훈련 데이터에 너무 맞춰져서 새로운 데이터에 대한 예측 능력이 떨어지는 과적합 문제를 해결하기 위해 사용됩니다. 모델 정규화는 모델의 복잡도를 조절하고, 가중치를 최적화하여 모델이 더 일반화되고 안정적으로 예측을 수행할 수 있도록 돕는 기술입니다.

In [27]:
async for chunk in chat.astream_log(messages):
    print(chunk)

RunLogPatch({'op': 'replace',
  'path': '',
  'value': {'final_output': None,
            'id': '009f564b-8e06-4330-ab8f-3cad8ec70c4e',
            'logs': {},
            'name': 'ChatOpenAI',
            'streamed_output': [],
            'type': 'llm'}})
RunLogPatch({'op': 'add',
  'path': '/streamed_output/-',
  'value': AIMessageChunk(content='')},
 {'op': 'replace',
  'path': '/final_output',
  'value': AIMessageChunk(content='')})
RunLogPatch({'op': 'add',
  'path': '/streamed_output/-',
  'value': AIMessageChunk(content='모')},
 {'op': 'replace',
  'path': '/final_output',
  'value': AIMessageChunk(content='모')})
RunLogPatch({'op': 'add',
  'path': '/streamed_output/-',
  'value': AIMessageChunk(content='델')},
 {'op': 'replace',
  'path': '/final_output',
  'value': AIMessageChunk(content='모델')})
RunLogPatch({'op': 'add',
  'path': '/streamed_output/-',
  'value': AIMessageChunk(content=' 정')},
 {'op': 'replace',
  'path': '/final_output',
  'value': AIMessageChunk(content='모델 정

#### Caching 
LangChain은 Chat Model을 위한 캐싱 레이어를 옵션으로 제공합니다. 
- 동일한 completion을 여러 번 요청하는 경우 LLM 제공업체에 대한 API 호출 횟수를 줄여 비용을 절감할 수 있습니다.
- LLM 제공업체에 대한 API 호출 횟수를 줄임으로써 애플리케이션 속도를 높일 수 있습니다.

In [28]:
from langchain.globals import set_llm_cache
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()

In [29]:
%%time
from langchain.cache import InMemoryCache

set_llm_cache(InMemoryCache())

# The first time, it is not yet in cache, so it should take longer
llm.predict("Tell me a joke")

d:\강의자료\생성형AI로AI애플리케이션개발하기\py311\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `predict` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


CPU times: total: 359 ms
Wall time: 1.29 s


"Why couldn't the bicycle stand up by itself? Because it was two tired!"

In [30]:
%%time
# The second time it is, so it goes faster
llm.predict("Tell me a joke")

CPU times: total: 0 ns
Wall time: 1.99 ms


"Why couldn't the bicycle stand up by itself? Because it was two tired!"

#### SQLite Cache

In [31]:
# !rm .langchain.db
!del .langchain.db

In [32]:
# We can do the same thing with a SQLite cache
from langchain.cache import SQLiteCache

set_llm_cache(SQLiteCache(database_path=".langchain.db"))

In [33]:
%%time
# The first time, it is not yet in cache, so it should take longer
llm.predict("Tell me a joke")

CPU times: total: 15.6 ms
Wall time: 21.4 s


"Why couldn't the bicycle stand up by itself?\n\nBecause it was two tired!"

In [34]:
%%time
# The second time it is, so it goes faster
llm.predict("Tell me a joke")

CPU times: total: 0 ns
Wall time: 3 ms


"Why couldn't the bicycle stand up by itself?\n\nBecause it was two tired!"

#### Function calling 
점점 더 많은 채팅 모델들이 OpenAI, Gemini 등과 같이 함수와 그 인수를 설명하고,    
호출할 함수와 해당 함수에 대한 입력이 포함된 JSON 객체를 반환하도록 하는 함수 호출 API를 갖추고 있습니다.   
함수 호출은 도구를 사용하는 체인과 에이전트를 구축하고, 보다 일반적으로 모델에서 구조화된 출력을 얻는 데 매우 유용합니다.  
  
LangChain에는 함수 호출을 쉽게 할 수 있는 여러 유틸리티가 함께 제공됩니다.  
- 함수를 모델에 바인딩하기 위한 간단한 구문  
- 다양한 유형의 객체를 예상되는 함수 스키마로 포맷하기 위한 변환기   
- API 응답에서 함수 호출을 추출하기 위한 출력 파서  
- 함수 호출을 기반으로 구축된 모델에서 구조화된 출력을 얻기 위한 체인  

많은 모델이 다양한 함수형 객체의 형식을 지정하고 모델에 바인딩하는 작업을 처리하는  helper method를 구현합니다.  
다음 Pydantic function schema를 가지고 다른 모델이 이를 호출하도록 하는 방법을 살펴보겠습니다. 

In [35]:
%pip install -qU langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [36]:
from langchain_core.pydantic_v1 import BaseModel, Field


# Note that the docstrings here are crucial, as they will be passed along
# to the model along with the class name.
class Multiply(BaseModel):
    """Multiply two integers together."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")

In [37]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
llm_with_tools = llm.bind_tools([Multiply])
llm_with_tools.invoke("what's 3 * 12")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_f88WsLKWcmsZlMcJlTnWeIWw', 'function': {'arguments': '{"a":3,"b":12}', 'name': 'Multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 62, 'total_tokens': 80}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': 'fp_fa89f7a861', 'finish_reason': 'tool_calls', 'logprobs': None})

도구 파서를 추가하여 생성된 메시지에서 도구 호출을 JSON으로 추출할 수 있습니다:

In [38]:
from langchain_core.output_parsers.openai_tools import JsonOutputToolsParser

tool_chain = llm_with_tools | JsonOutputToolsParser()
tool_chain.invoke("what's 3 * 12")

[{'type': 'Multiply', 'args': {'a': 3, 'b': 12}}]

또는 원래의 피단틱 클래스로 돌아갈 수도 있습니다:

In [39]:
from langchain_core.output_parsers.openai_tools import PydanticToolsParser

tool_chain = llm_with_tools | PydanticToolsParser(tools=[Multiply])
tool_chain.invoke("what's 3 * 12")

[Multiply(a=3, b=12)]

#### 특정 도구가 강제로 사용되도록(그리고 한 번만 사용되도록) 하려면 tool_choice 인수를 설정하면 됩니다.

In [40]:
llm_with_multiply = llm.bind_tools([Multiply], tool_choice="Multiply")
llm_with_multiply.invoke(
    "make up some numbers if you really want but I'm not forcing you"
)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_w5nfVv8FpgsL9VIyQTMj09qv', 'function': {'arguments': '{"a":5,"b":10}', 'name': 'Multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 78, 'total_tokens': 87}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': 'fp_4f0b692a78', 'finish_reason': 'stop', 'logprobs': None})

#### 토큰 사용량 추적

In [41]:
from langchain.callbacks import get_openai_callback
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()
with get_openai_callback() as cb:
    result = llm.invoke("Tell me a funny joke")
    print(cb)

Tokens Used: 28
	Prompt Tokens: 12
	Completion Tokens: 16
Successful Requests: 1
Total Cost (USD): $4.9999999999999996e-05


### LLM
- LLM은 LangChain의 핵심 구성 요소입니다.
- LangChain은 OpenAI, Cohere, Hugging Face 등에서 제공하는 다양한 LLM과 상호 작용하기 위한 표준 인터페이스를 제공합니다.
  
- LLM은 랭체인 표현식 언어(LCEL)의 기본 빌딩 블록인 Runnable  인터페이스를 구현합니다.
- 즉, invoke, ainvoke, stream, astream, batch, abatch, astream_log 호출을 지원합니다.
- LLM은 문자열을 입력으로 받거나 List[BaseMessage] 및 PromptValue와 같이 문자열 프롬프트로 강제 변환할 수 있는 객체를 입력으로 받습니다.

In [42]:
from langchain_openai import OpenAI

llm = OpenAI()

In [43]:
llm.invoke(
    "실업률과 인플레이션의 관계에 대한 이론에는 어떤 것이 있나요?"
)

'\n\n1. 필립스 곡선 (Phillips Curve)\n필립스 곡선은 뉴질랜드의 경제학자 알반 필립스가 1958년에 발표한 이론으로, 실업률과 물가 상승률 사이의 역관계를 설명하는 곡선이다. 이론에 따르면, 실업률이 낮을수록 임금이 상승하고 생산성이 증가하므로 물가 상승률이 높아지는 경향이 있다.\n\n2. 나이어보-데미지 모델 (Nairu-Damage Model)\n나이어보-데미지 모델은 미국의 경제학자 데미지와 나이어보가 제시한 이론으로, 실업률이 자연실업률보다 높은 경우 장'

In [44]:
for chunk in llm.stream(
    "실업률과 인플레이션의 관계에 대한 이론에는 어떤 것이 있나요?"
):
    print(chunk, end="", flush=True)



1. 필립스 곡선 (Phillips Curve) 이론
- 뉴질랜드 출신 경제학자 알번 윌리엄 필립스가 제안한 이론으로, 물가상승률과 실업률 사이의 관계를 설명한다. 이 이론에 따르면 실업률이 낮으면 임금 상승률이 높아지고 이는 물가 상승률을 야기한다.

2. 자연유리실업률 이론 (Natural Rate of Unemployment Theory)
- 미국의 경제학자 에드워드 프레스콧과 노벨 경제학자 로버트 뫼디글리아니가 제안한 이론으로, 실업률이 자연유리실업률 이하로 유지되면 인플레이션은

In [45]:
await llm.ainvoke(
    "실업률과 인플레이션의 관계에 대한 이론에는 어떤 것이 있나요?"
)

'\n\n1. 필립스 곡선 (Phillips Curve)\n필립스 곡선은 뉴질랜드의 경제학자 알반 필립스가 1958년에 발표한 이론으로, 실업률과 물가 상승률 사이의 역관계를 설명하는 곡선이다. 이론에 따르면, 실업률이 낮을수록 임금이 상승하고 생산성이 증가하므로 물가 상승률이 높아지는 경향이 있다.\n\n2. 나이어보-데미지 모델 (Nairu-Damage Model)\n나이어보-데미지 모델은 미국의 경제학자 데미지와 나이어보가 제시한 이론으로, 실업률이 자연실업률보다 높은 경우 장'

In [46]:
async for chunk in llm.astream(
    "실업률과 인플레이션의 관계에 대한 이론에는 어떤 것이 있나요?"
):
    print(chunk, end="", flush=True)



1. 필립스 곡선 (Phillips Curve) 이론
- 이론의 개요: 이론은 뉴질랜드의 경제학자 윌리엄 필립스에 의해 제안된 것으로, 실업률과 물가 상승률 사이에 반비례 관계가 있다는 것을 설명한다. 즉, 실업률이 낮으면 물가 상승률이 높아지고, 실업률이 높으면 물가 상승률이 낮아지는 경향을 보인다는 것이다. 이는 실업률이 높을 때 노동자들의 협상력이 약해지고 임금이 낮아지기 때문에 생산비용이 낮아지고, 그 결과로 물가 상승률이 낮아지는 것으로 설명된다.

2. 자연

In [47]:
await llm.abatch(
    [
        "실업률과 인플레이션의 관계에 대한 이론에는 어떤 것이 있나요?"
    ]
)

['\n\n1. 필립스 곡선 (Phillips Curve)\n필립스 곡선은 뉴질랜드의 경제학자 알반 필립스가 1958년에 발표한 이론으로, 실업률과 물가 상승률 사이의 역관계를 설명하는 곡선이다. 이론에 따르면, 실업률이 낮을수록 임금이 상승하고 생산성이 증가하므로 물가 상승률이 높아지는 경향이 있다.\n\n2. 나이어보-데미지 모델 (Nairu-Damage Model)\n나이어보-데미지 모델은 미국의 경제학자 데미지와 나이어보가 제시한 이론으로, 실업률이 자연실업률보다 높은 경우 장']

### Output parsers
- Output parser(출력 구문 분석기)로 언어모델의 텍스트 출력을 구조화 합니다.
- Output parser에는 다음과 같은 몇 가지 주요 유형이 있습니다:
- LLM의 텍스트를 구조화된 정보(예: JSON)로 변환
- ChatMessage를 단순한 문자열로 변환
- 메시지 외에 호출에서 반환된 추가 정보(예: OpenAI 함수 호출)를 문자열로 변환

#### CommaSeparatedListOutputParser

In [48]:
from langchain.output_parsers import CommaSeparatedListOutputParser

output_parser = CommaSeparatedListOutputParser()
output_parser.parse("hi, bye")

['hi', 'bye']

#### DatetimeOutputParser

In [49]:
from langchain.output_parsers import DatetimeOutputParser
from langchain.prompts import PromptTemplate
from langchain_openai import OpenAI

output_parser = DatetimeOutputParser()
template = """Answer the users question:

{question}

{format_instructions}"""
prompt = PromptTemplate.from_template(
    template,
    partial_variables={"format_instructions": output_parser.get_format_instructions()},
)

prompt

PromptTemplate(input_variables=['question'], partial_variables={'format_instructions': "Write a datetime string that matches the following pattern: '%Y-%m-%dT%H:%M:%S.%fZ'.\n\nExamples: 1903-10-29T13:23:14.114723Z, 1137-06-22T20:20:35.995796Z, 0426-12-12T21:21:42.127985Z\n\nReturn ONLY this string, no other words!"}, template='Answer the users question:\n\n{question}\n\n{format_instructions}')

In [50]:
chain = prompt | OpenAI() | output_parser
output = chain.invoke({"question": "when was bitcoin founded?"})

print(output)

2009-01-03 00:00:00


#### Pydantic parser
- 사용자가 임의의 Pydantic Model을 지정하고 해당 스키마를 준수하는 출력을 위해 LLM을 쿼리할 수 있다.
- Pydantic은 이터 유효성 검사와 구조화된 데이터를 파싱하기 위한 라이브러리입니다. 

In [51]:
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain_openai import OpenAI

model = OpenAI(model_name="gpt-3.5-turbo-instruct", temperature=0.0)


# Define your desired data structure.
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")

    # You can add custom validation logic easily with Pydantic.
    @validator("setup")
    def question_ends_with_question_mark(cls, field):
        if field[-1] != "?":
            raise ValueError("Badly formed question!")
        return field


# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# And a query intended to prompt a language model to populate the data structure.
prompt_and_model = prompt | model
output = prompt_and_model.invoke({"query": "Tell me a joke."})
parser.invoke(output)

Joke(setup='Why did the tomato turn red?', punchline='Because it saw the salad dressing!')

##### LCEL
- Output parser는 [LCEL, LangChain Expression Language](https://python.langchain.com/docs/expression_language/) 기본 구성 요소인  [Runnable interface](https://python.langchain.com/docs/expression_language/interface) 를 구현합니다.
- 즉, invoke, ainvoke, stream, astream, batch, abatch, astream_log 호출을 지원합니다.
- Output parser는 문자열 또는 BaseMessage를 입력으로 받아들이며 임의의 유형을 반환할 수 있습니다.

In [52]:
parser.invoke(output)

Joke(setup='Why did the tomato turn red?', punchline='Because it saw the salad dressing!')

Output parser를 수동으로 호출하는 대신 Runnable 시퀀스에 추가할 수도 있습니다.

In [53]:
chain = prompt | model | parser
chain.invoke({"query": "Tell me a joke."})

Joke(setup='Why did the tomato turn red?', punchline='Because it saw the salad dressing!')

모든 파서가 스트리밍 인터페이스를 지원하지만, 특정 파서만 부분적으로 파싱된 객체를 통해 스트리밍할 수 있는데,   
이는 출력 유형에 따라 크게 달라지기 때문입니다. 부분 객체를 구성할 수 없는 파서는 단순히 완전히 파싱된 출력을 생성합니다.  
    
	
예를 들어 SimpleJsonOutputParser는 부분 출력을 통해 스트리밍할 수 있습니다:

In [54]:
from langchain.output_parsers.json import SimpleJsonOutputParser

json_prompt = PromptTemplate.from_template(
    "Return a JSON object with an `answer` key that answers the following question: {question}"
)
json_parser = SimpleJsonOutputParser()
json_chain = json_prompt | model | json_parser

In [55]:
list(json_chain.stream({"question": "누가 현미경을 누가 발명했나요?"}))

[{},
 {'answer': ''},
 {'answer': '안'},
 {'answer': '안토'},
 {'answer': '안토니'},
 {'answer': '안토니 반'},
 {'answer': '안토니 반 레'},
 {'answer': '안토니 반 레웨'},
 {'answer': '안토니 반 레웨른'},
 {'answer': '안토니 반 레웨른후'},
 {'answer': '안토니 반 레웨른후크'},
 {'answer': '안토니 반 레웨른후크가'},
 {'answer': '안토니 반 레웨른후크가 현'},
 {'answer': '안토니 반 레웨른후크가 현미'},
 {'answer': '안토니 반 레웨른후크가 현미경'},
 {'answer': '안토니 반 레웨른후크가 현미경을'},
 {'answer': '안토니 반 레웨른후크가 현미경을 발'},
 {'answer': '안토니 반 레웨른후크가 현미경을 발명'},
 {'answer': '안토니 반 레웨른후크가 현미경을 발명했습니다'},
 {'answer': '안토니 반 레웨른후크가 현미경을 발명했습니다.'}]